In [1]:
import pdfplumber
from transformers import pipeline
from huggingface_hub import hf_hub_download
import json
import pandas as pd

# Step 1: Extract Text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text


In [3]:
# Step 3: Process the PDF and Generate Code
pdf_path = "C:/Users/lnpka/projects/Chat-With-CSV-Using-Agent/Regulatory_Repoting_Instructions.pdf"
rules_text = extract_text_from_pdf(pdf_path)
generated_code = generate_validation_functions(rules_text)

# Step 4: Save and Execute the Generated Code
if generated_code:
    generated_script_path = "validation_rules.py"
    with open(generated_script_path, "w") as file:
        file.write(generated_code)
    try:
        exec(generated_code)  # Dynamically define validation functions
    except Exception as e:
        print(f"Error executing generated code: {e}")

# Step 5: Example Transaction Validation (Assuming LLM generates functions like validate_transaction_amount)
def validate_transaction(transaction):
    results = {}
    
    if 'validate_transaction_amount' in globals():
        results['Transaction Amount'] = validate_transaction_amount(transaction['Transaction_Amount'], transaction['Reported_Amount'], transaction.get('Is_Cross_Currency', False))
    
    if 'validate_account_balance' in globals():
        results['Account Balance'] = validate_account_balance(transaction['Account_Balance'], transaction.get('OD_Flag', False))
    
    return results

# Step 6: Read Transactions from CSV and Validate
csv_path = "C:/Users/lnpka/projects/Chat-With-CSV-Using-Agent/customer_transactions.csv"
try:
    df = pd.read_csv(csv_path)
    validation_results = []
    for _, row in df.iterrows():
        transaction = row.to_dict()
        validation_results.append(validate_transaction(transaction))
    
    # Print results
    print(json.dumps(validation_results, indent=4))
except Exception as e:
    print(f"Error reading CSV file: {e}")

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

D:\Sudha\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\lnpka\.cache\huggingface\hub\models--meta-llama--Llama-2-7b-chat-hf. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

D:\Sudha\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:983: UserWarning: Not enough free disk space to download the file. The expected file size is: 9976.58 MB. The target location C:\Users\lnpka\.cache\huggingface\hub\models--meta-llama--Llama-2-7b-chat-hf\blobs only has 6258.80 MB free disk space.
  warnings.warn(


model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

D:\Sudha\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:983: UserWarning: Not enough free disk space to download the file. The expected file size is: 9976.58 MB. The target location C:\Users\lnpka\.cache\huggingface\hub\models--meta-llama--Llama-2-7b-chat-hf\blobs only has 0.00 MB free disk space.
  warnings.warn(


model-00001-of-00002.safetensors:  63%|######2   | 6.27G/9.98G [00:00<?, ?B/s]

Error loading LLM model: Could not load model meta-llama/Llama-2-7b-chat-hf with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 'transformers.models.auto.modeling_tf_auto.TFAutoModelForCausalLM'>, <class 'transformers.models.llama.modeling_llama.LlamaForCausalLM'>). See the original errors:

while loading with AutoModelForCausalLM, an error is thrown:
Traceback (most recent call last):
  File "D:\Sudha\anaconda3\Lib\site-packages\transformers\pipelines\base.py", line 286, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Sudha\anaconda3\Lib\site-packages\transformers\models\auto\auto_factory.py", line 564, in from_pretrained
    return model_class.from_pretrained(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Sudha\anaconda3\Lib\site-packages\transformers\modeling_utils.py", line 3715, in from_pretrained
    resolved

In [8]:
# Step 2: Manually Define Validation Functions
def validate_transaction_amount(transaction_amount, reported_amount, is_cross_currency):
    #if is_cross_currency:
     #   return abs(transaction_amount - reported_amount) <= 0.01 * reported_amount
    return transaction_amount == reported_amount

def validate_account_balance(account_balance, od_flag):
    return account_balance >= 0 or od_flag

def validate_currency(currency):
    valid_currencies = {"USD", "EUR", "INR", "GBP", "JPY"}  # Example ISO 4217 codes
    return currency in valid_currencies

def validate_country(country):
    non_accepted_countries = {"North Korea", "Iran", "Syria", "Sudan", "Cuba", "Venezuela"}  # Example valid countries
    return country not in non_accepted_countries

def validate_transaction_date(transaction_date):
    from datetime import datetime, timedelta
    today = datetime.today()
    date_format = "%Y-%m-%d"  # Adjust format as per CSV
    try:
        tx_date = datetime.strptime(transaction_date, date_format)
        return tx_date <= today and (today - tx_date).days <= 365
    except ValueError:
        return False

def calculate_risk_score(transaction):
    risk_score = 0
    
    # Infer cross-currency condition
    is_cross_currency = transaction["Transaction_Currency"] != transaction["Customer_Currency"]
    
    # Check individual validation rules
    if not validate_transaction_amount(transaction["Transaction_Amount"], transaction["Reported_Amount"], is_cross_currency):
        risk_score += 1  # Low risk
    
    if not validate_account_balance(transaction["Account_Balance"], transaction.get("OD_Flag", False)):
        risk_score += 3  # Medium risk

    if not validate_currency(transaction["Transaction_Currency"]):
        risk_score += 3  # Medium risk
    
    if not validate_country(transaction["Country"]):
        risk_score += 1  # Low risk
    
    if not validate_transaction_date(transaction["Transaction_Date"]):
        risk_score += 3  # Medium risk

    # High-risk country & high amount
    high_risk_countries = {"North Korea", "Iran", "Syria", "Sudan", "Cuba", "Venezuela"}  # Add actual high-risk country codes
    if transaction["Country"] in high_risk_countries and transaction["Transaction_Amount"] > 5000:
        risk_score += 5  # High risk

    # Round-number transaction (possible money laundering)
    if transaction["Transaction_Amount"] in {1000, 5000, 10000}:
        risk_score += 3  # Medium risk
    
    # Cross-border transaction > $10,000 without remarks
    if transaction["Transaction_Amount"] > 10000 and "Remarks" not in transaction:
        risk_score += 3  # Medium risk

    return min(risk_score, 5)  # Cap risk at 5 (max risk level)

def validate_transaction(transaction):
    results = {
        "Transaction Amount": validate_transaction_amount(transaction["Transaction_Amount"], transaction["Reported_Amount"], transaction["Transaction_Currency"] != transaction["Customer_Currency"]),
        "Account Balance": validate_account_balance(transaction["Account_Balance"], transaction.get("OD_Flag", False)),
        "Currency": validate_currency(transaction["Transaction_Currency"]),
        "Country": validate_country(transaction["Country"]),
        "Transaction Date": validate_transaction_date(transaction["Transaction_Date"]),
        "Risk Score": calculate_risk_score(transaction),  # Add risk score calculation
    }
    return results

# Step 6: Read Transactions from CSV and Validate
csv_path = "C:/Users/lnpka/projects/Chat-With-CSV-Using-Agent/customer_transactions1.csv"
try:
    df = pd.read_csv(csv_path)
    validation_results = []
    for _, row in df.iterrows():
        transaction = row.to_dict()
        validation_results.append(validate_transaction(transaction))
    
    # Print results
    print(json.dumps(validation_results, indent=4))
except Exception as e:
    print(f"Error reading CSV file: {e}")



[
    {
        "Transaction Amount": true,
        "Account Balance": true,
        "Currency": true,
        "Country": true,
        "Transaction Date": false,
        "Risk Score": 3
    },
    {
        "Transaction Amount": true,
        "Account Balance": true,
        "Currency": true,
        "Country": true,
        "Transaction Date": false,
        "Risk Score": 3
    },
    {
        "Transaction Amount": true,
        "Account Balance": true,
        "Currency": true,
        "Country": true,
        "Transaction Date": false,
        "Risk Score": 3
    },
    {
        "Transaction Amount": true,
        "Account Balance": false,
        "Currency": true,
        "Country": true,
        "Transaction Date": false,
        "Risk Score": 5
    },
    {
        "Transaction Amount": true,
        "Account Balance": true,
        "Currency": true,
        "Country": true,
        "Transaction Date": false,
        "Risk Score": 3
    },
    {
        "Transaction Amount": t

In [ ]:
# Step 4: Read CSV and Validate Transactions
csv_path = "C:/Users/lnpka/projects/Chat-With-CSV-Using-Agent/customer_transactions.csv"
try:
    df = pd.read_csv(csv_path)
    validation_results = [validate_transaction(row.to_dict()) for _, row in df.iterrows()]
    print(json.dumps(validation_results, indent=4))
except Exception as e:
    print(f"Error reading CSV file: {e}")

In [1]:
!free -h  # Check available RAM
!nvidia-smi 

'free' is not recognized as an internal or external command,
operable program or batch file.
'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.
